<span style="font-size:24pt; color:blue; font-family: 'Times New Roman'">P-box Notebook 
    <span style="color:green;"> for creating AR6 style P-box data </span>
</span>
<br>
<span style="font-size:12pt; color:black; font-family:Georgia, serif;font-style:italic">by Praveen Kumar and Robert Kopp.</span>
<br>
<br>
<span style="font-family: 'Times New Roman'; font-size:16pt; color:black"> 
    This is a notebook that can be used to create and vizualize P-box data from raw <a href="https://doi.org/10.5194/egusphere-2023-14" style="color:blue; text-decoration:underline;">FACTS 1.0</a>
    output. Workflows and associated *.nc files are created as part of this notebook. 
<br>
<br>
<span style="font-family:'Courier New', monospace; font-size:12pt; color:black"> <b>Background::</b> <br>
    In order to represent deep uncertainty, Probability boxes (p-boxes) can be constructed. These represent an envelope encompassing alternative probability distributions for describing the same variables. Within this notebook, we construct four p-boxes: <strong>1e, 1f, 2e,</strong> and <strong>2f</strong>.
    <br><br>
    <strong>P-box 1e</strong> and <strong>2e</strong> use ice-sheet projections from an emulator that accurately captures the output of both the GlacierMIP glacier model intercomparison and the ISMIP6 ice-sheet model intercomparison.
    <br><br>
    <strong>P-box 1e</strong> and <strong>1f</strong> both use multi-model intercomparison exercises 
    (<a href="https://www.nature.com/articles/s41586-021-03302-y" style="color:blue; text-decoration:underline;">ISMIP6</a> and <a href="https://doi.org/10.5194/esd-11-35-2020" style="color:blue; text-decoration:underline;">LARMIP2</a>). 
    These p-boxes represent a medium-to-high level of agreement among the participating models. The IPCC AR6 thus assessed medium confidence in their projections of included processes.
</span>
<br>
<br>
<br>

<br>
<span style="font-size:24pt; color:blue; font-family: 'Times New Roman';"> Set <b>expFolder</b> ::
    <span style="font-size:14pt; color:green;"> point to where FACTS raw output is located (e.g. /facts/experiments) </span>

</span>


<span style="background-color: yellow;"> need to modify for NZ and generic facts experiments </span> currently configured to the coupling syntax

In [1]:
expFolder="/projects/kopp/facts-experiments/221217"
#
region = 'global'
#
import fun_pbox_Gen_v5 as fn
#
ssps=["ssp119","ssp126","ssp245","ssp370","ssp585"]
import os

<br>
<span style="font-size:24pt; color:blue; font-family: 'Times New Roman';"> Generate::
    <span style="font-size:14pt; color:green;">Quantile files from FACTS output and arrange in workflow/ssp folders </span>

</span>


In [2]:
fn.generate_pbox_nc_files_(expFolder,region,ssps,"1_workflow")

<br>
<span style="font-size: 32pt; color: red; font-family: 'Times New Roman'"> 
    Generate:: <br>
    <ol style="font-size: 12pt; color: green; font-family: 'Times New Roman'">
    <li> Pbox files and folders</li>
</ol>  
</span>

In [5]:
configurations = {
            'pb_1e': {
                'files_to_copy':    [('wf2e', 'larmip.AIS'), ('wf2e', 'total.workflow')],
                'AIS_pb':           ["emulandice.AIS", "larmip.AIS"],
                'total_pb':         ["total.workflow.wf1e", "total.workflow.wf2e"]
            },
            'pb_1f': {
                'files_to_copy':    [('wf2f', 'larmip.AIS'), ('wf2f', 'total.workflow')],
                'AIS_pb':           ["ipccar5.icesheets_AIS", "larmip.AIS"],
                'total_pb':         ["total.workflow.wf1f", "total.workflow.wf2f"]
            },
            'pb_2e': {
                'files_to_copy':    [('wf1e', 'emulandice.AIS'), ('wf3e', 'deconto21.AIS'), ('wf4', 'bamber19.icesheets_AIS'), 
                                     ('wf4', 'bamber19.icesheets_GIS'), ('wf1f', 'ipccar5.glaciers'),
                                     ('wf1e', 'total.workflow'), ('wf3e', 'total.workflow'), ('wf4', 'total.workflow')],
                'AIS_pb':           ["emulandice.AIS", "larmip.AIS", "deconto21.AIS", "bamber19.icesheets_AIS"],
                'GIS_pb':           ["emulandice.GrIS", "bamber19.icesheets_GIS"],
                'glacier_pb':       ["emulandice.glaciers", "ipccar5.glaciers"],
                'total_pb':         ["total.workflow.wf1e", "total.workflow.wf2e", "total.workflow.wf3e", "total.workflow.wf4"]
            },
            'pb_2f': {
                'files_to_copy':    [('wf1f', 'ipccar5.icesheets_AIS*'), ('wf3f', 'deconto21.AIS*'), 
                                     ('wf4', 'bamber19.icesheets_AIS*'), ('wf4', 'bamber19.icesheets_GIS')
                                     ('wf1f', 'total.workflow'), ('wf3f', 'total.workflow'),('wf4', 'total.workflow')],
                'AIS_pb':           ["ipccar5.icesheets_AIS", "larmip.AIS", "deconto21.AIS", "bamber19.icesheets_AIS"],
                'GIS_pb':           ["GrIS1f.FittedISMIP.GrIS", "bamber19.icesheets_GIS"],
                'total_pb':         ["total.workflow.wf1f", "total.workflow.wf2f", "total.workflow.wf3f", "total.workflow.wf4"]
            }
}
#
copy_file_with_pattern = lambda ssp, wf, pattern: fn.copy_filename_with_pattern(f"{os.getcwd()}/1_workflow/{wf}/{ssp}", ssp_path, f"*{pattern}*")
copy_all_wf_files = lambda p, s, d: fn.copy_all_files_from(os.path.join(os.getcwd(), f'1_workflow/wf{p.split("_")[1]}/{s}'), d)
#    
def create_pb_files(ssp_path, patterns, outfile_prefix, pbox, ssp):
    infiles = [f'{ssp_path}/{fn.find_filename_with_pattern(ssp_path, pattern)}' for pattern in patterns]
    outfile = f"{ssp_path}/{outfile_prefix}-pb{pbox.split('_')[1]}-{ssp}_globalsl.nc"
    fn.generate_pbox(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
    #
#
path = fn.create_directory("2_pbox","remove")
#
for pbox in fn.PB_file_patterns:
    fn.create_directory(f'{os.path.join(path, pbox)}')
    for ssp in ssps:
        # ..............................................................................
        if pbox in ['pb_2e', 'pb_2f'] and ssp in['ssp119','ssp245','ssp370']: continue
        # ..............................................................................
        ssp_path = fn.create_directory(f'{os.path.join(path, pbox,ssp)}')
        #
        # ---> 
        if pbox in configurations:
            config = configurations[pbox]
            copy_all_wf_files(pbox, ssp, ssp_path)
            #
            for indv_wf, pattern in config['files_to_copy']:
                copy_file_with_pattern(ssp, indv_wf, pattern)
                
            #
            create_pb_files(ssp_path, config['AIS_pb'], "icesheets-AIS", pbox, ssp)
            create_pb_files(ssp_path, config['total_pb'], "total-workflow", pbox, ssp)
            if pbox not in ['pb_1e','pb_1f']:
                create_pb_files(ssp_path, config['GIS_pb'], "icesheets-GIS", pbox, ssp)
                if pbox in ['pb_2e']: create_pb_files(ssp_path, config['glacier_pb'], "glaciers", pbox, ssp)


            # Uncomment below if you want to delete files
            for pattern1, pattern2 in [("*AIS*", "icesheets-AIS-pb*"), ("*GIS*", "icesheets-GIS-pb*"), ("*glaciers*", "glaciers-pb*"), ("*total*", "*total-workflow-pb*")]:
                fn.delete_files_with_pattern(ssp_path, pattern1, pattern2)
            
        

<br>
<span style="font-size: 32pt; color: red; font-family: 'Times New Roman'"> 
    Generate:: <br>
    <ol style="font-size: 12pt; color: green; font-family: 'Times New Roman'">
    <li> Confidence level files files and folders</li>
</ol>  
</span>

In [4]:
path = fn.create_directory("3_confidence_level_files","remove")
fn.generate_confidence_files(os.getcwd()+'/2_pbox', path)